In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time

In [2]:
class Net(nn.Module):

    def __init__(self, grouped=False):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        if grouped:
            self.conv1 = torch.nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3, groups=16)
        else:
            self.conv1 = torch.nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = self.conv1(x)
        return x

In [3]:
x=torch.rand(7,64,32,32).cuda()
net=Net(grouped=False)
net.cuda()
torchinput = torch.autograd.Variable(x).cuda()

begin=time.time()
for i in range(100):
    net(torchinput)
print time.time()-begin

net=Net(grouped=True)
net.cuda()

begin=time.time()
for i in range(100):
    net(torchinput)
print time.time()-begin

1.03984498978
0.0877940654755


In [4]:
import mxnet as mx

/home/tapir/miniconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
data=mx.sym.Variable('data')
ctx=mx.gpu()

sym1=mx.sym.Convolution(data, num_filter=128, kernel=(3,3))
sym2=mx.sym.Convolution(data, num_filter=1024, kernel=(3,3), num_group=16)

img = img=mx.nd.random.uniform(0, 1, shape=(7,64,32,32))
data_iter = mx.io.NDArrayIter(img, batch_size= 7, label_name=None)

mod1=mx.mod.Module(symbol=sym1, context=ctx)
mod2=mx.mod.Module(symbol=sym2, context=ctx)

mod1.bind( data_shapes=data_iter.provide_data)
mod2.bind( data_shapes=data_iter.provide_data)

mod1.init_params()
mod2.init_params()

/home/tapir/miniconda2/lib/python2.7/site-packages/mxnet/module/base_module.py:54: UserWarning: You created Module with Module(..., label_names=['softmax_label']) but input with name 'softmax_label' is not found in symbol.list_arguments(). Did you mean one of:
	data
  warnings.warn(msg)
/home/tapir/miniconda2/lib/python2.7/site-packages/mxnet/module/base_module.py:66: UserWarning: Data provided by label_shapes don't match names specified by label_names ([] vs. ['softmax_label'])
  warnings.warn(msg)


In [6]:
begin=time.time()
for i in range(100):
    mod1.predict(data_iter).asnumpy()
print time.time()-begin


begin=time.time()
for i in range(100):
    mod2.predict(data_iter).asnumpy()
print time.time()-begin


0.884229898453
3.17296814919


In [34]:
class Net(nn.Module):

    def __init__(self,inch, double=False):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        if double:
            self.conv1 = torch.nn.Conv2d(in_channels=inch,out_channels=2*inch,kernel_size=3)
        else:
            self.conv1 = torch.nn.Conv2d(in_channels=inch,out_channels=inch,kernel_size=3)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = self.conv1(x)
        return x

In [36]:
x=torch.rand(7,64,32,32)
net=Net(x.shape[1],double=False)
net
torchinput = torch.autograd.Variable(x)

begin=time.time()
for i in range(100):
    res=net(torchinput).data.numpy()
print time.time()-begin

print res.shape

net=Net(x.shape[1],double=True)

begin=time.time()
for i in range(100):
    res= net(torchinput).data.numpy()
print time.time()-begin
print res.shape


0.947057962418
(7, 64, 30, 30)
1.63933706284
(7, 128, 30, 30)


In [37]:
import mxnet as mx
data=mx.sym.Variable('data')
ctx=mx.cpu()

sym1=mx.sym.Convolution(data, num_filter=64, kernel=(3,3))
sym2=mx.sym.Convolution(data, num_filter=128, kernel=(3,3))

img = img=mx.nd.random.uniform(0, 1, shape=(7,64,32,32))
data_iter = mx.io.NDArrayIter(img, batch_size= 7, label_name=None)

mod1=mx.mod.Module(symbol=sym1, context=ctx)
mod2=mx.mod.Module(symbol=sym2, context=ctx)

mod1.bind( data_shapes=data_iter.provide_data)
mod2.bind( data_shapes=data_iter.provide_data)

mod1.init_params()
mod2.init_params()

In [38]:
begin=time.time()
for i in range(100):
    res1=mod1.predict(data_iter).asnumpy()
print time.time()-begin

print res1.shape

begin=time.time()
for i in range(100):
    res2=mod2.predict(data_iter).asnumpy()
print time.time()-begin

print res2.shape

1.59898805618
(7, 64, 30, 30)
2.50224304199
(7, 128, 30, 30)
